In [1]:
from IPython.core.display import HTML
css_file = '../../../../msc/custom.css' 
HTML(open(css_file, 'r').read()) 

# The Generalized Roy Model


In [2]:
# standard library
import os
import sys
import numpy as np

# edit PYTHONPATH
sys.path.insert(0, 'grmpy')

# project library
from tests._auxiliary import random_init
import grmpy as gp

First, let us illustrate simple class to the the functions of the package.

In [3]:
# Process initialization file
init_dict = gp.process('init.ini')

# Simulate synthetic sample
gp.simulate(init_dict)

# Estimate model
rslt = gp.estimate(init_dict)

# Write results
gp.inspect(rslt, init_dict)

# Inspect the results
%cat results.txt


 softEcon: Generalized Roy Model
 -------------------------------

 Average Treatment Effects

     ATE       -0.41

     TT        -1.69

     TUT        0.87


 Parameters

     Start    Finish

      0.34      0.34
      0.55      0.55
      0.58      0.58
      0.52      0.52
      0.00      0.00
      0.99      0.99
      0.91      0.91
      0.21      0.21
      0.29      0.29
      0.52      0.52
      0.90      0.90
      0.98      0.98
      0.26      0.26
      0.06      0.06
      0.31      0.31


There are several implementations of the likelihood function available: fast, slow object. You can select the implementation to use in the initialization file. Let us make sure that the result is the same regardless of the implementation. 

In [4]:
NUM_TESTS = 1

for _ in range(NUM_TESTS):

    # Generate random request
    init_dict = random_init()

    # Ensure same starting value
    init_dict['ESTIMATION']['start'] = 'init'

    # Simulate sample
    gp.simulate(init_dict)

    # Estimate generalize Roy model
    rslt = dict()

    for version in ['slow', 'fast']:

        init_dict['ESTIMATION']['version'] = version

        rslt[version] = gp.estimate(init_dict)['fval']

    # Assert equality of results
    np.testing.assert_allclose(rslt['slow'], rslt['fast'])

    # Cleanup
    os.remove(init_dict['BASICS']['file'])

Now, let us finally have a look whether the program actually recovers the true structural parameters.

In [5]:
# Process initialization file
init_dict = gp.process('init.ini')

# Simulate synthetic sample
gp.simulate(init_dict)

# Estimate model
rslt = gp.estimate(init_dict)

# Write results
gp.inspect(rslt, init_dict)

# Inspect the results
%cat results.txt


 softEcon: Generalized Roy Model
 -------------------------------

 Average Treatment Effects

     ATE       -0.41

     TT        -1.69

     TUT        0.87


 Parameters

     Start    Finish

      0.34      0.34
      0.55      0.55
      0.58      0.58
      0.52      0.52
      0.00      0.00
      0.99      0.99
      0.91      0.91
      0.21      0.21
      0.29      0.29
      0.52      0.52
      0.90      0.90
      0.98      0.98
      0.26      0.26
      0.06      0.06
      0.31      0.31
